In [66]:
import numpy as np
import pandas as pd
from collections import OrderedDict, Counter
import itertools

In [67]:
dict_aggrageted_function_for_column = {
    'Energy_Balance': 'mean',
    'actual_allmeters': 'mean',
    'actual_pedmeters': 'mean',
    'bodymass': 'mean',
    'kcal_hr': 'mean',
    'locomotor_activity': 'mean',
    'rq': 'mean',
    'vco2': 'mean',
    'vo2': 'mean',
    'vh2o': 'mean',
    'xbreak': 'mean',
    'ybreak': 'mean',
    'actual_foodupa': 'sum',
    'actual_waterupa': 'sum',
}

add_feature_for_agg = {
  **dict_aggrageted_function_for_column, 
  'actual_allmeters_cumsum': 'mean',
  'actual_pedmeters_cumsum': 'mean'
}

In [68]:
df = pd.read_csv("all weeks shani expri.csv")

df[['vco2', 'vh2o', 'vo2']] = df[['vco2', 'vh2o', 'vo2']].mul(60)
df[['actual_foodupa']] = df[['actual_foodupa']].mul(3.56)
df['Energy_Balance'] = df['actual_foodupa'].values - df['kcal_hr'].values
df['locomotor_activity'] = df[['xbreak', 'ybreak']].sum(axis=1)

df

,Date_Time_1,subjectID,Group,actual_allmeters,actual_foodupa,actual_pedmeters,actual_waterupa,bodymass,envirolightlux,envirooccupancy,...,envirotemp,kcal_hr,rq,vco2,vh2o,vo2,xbreak,ybreak,Energy_Balance,locomotor_activity
0,2021-07-28 16:45:00,1,Control,0.820163,0.000000,0.259127,0.000000,24.46564,NaN,NaN,...,NaN,0.407835,0.758679,64.866900,8.167320,85.280940,79.0,122.0,-0.407835,201.0
1,2021-07-28 16:45:00,10,Control,1.404414,0.313275,1.140263,0.000000,22.61497,NaN,NaN,...,NaN,0.401331,0.818524,67.938600,9.748566,82.768560,64.0,147.0,-0.088056,211.0
2,2021-07-28 16:45:00,11,Group_3,3.553978,0.006243,3.171110,0.002538,21.34998,NaN,NaN,...,NaN,0.383572,0.794086,63.289920,8.856168,79.566900,150.0,210.0,-0.377329,360.0
3,2021-07-28 16:45:00,12,Group_2,11.915580,0.204500,11.116600,0.001420,24.69681,NaN,NaN,...,NaN,0.366636,0.843458,63.434640,7.688532,75.228840,191.0,597.0,-0.162136,788.0
4,2021-07-28 16:45:00,13,Control,3.425906,0.116263,2.752747,0.077571,25.82226,NaN,NaN,...,NaN,0.454166,0.772627,73.195680,8.843118,94.699680,11.0,429.0,-0.337903,440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46667,2021-09-09 12:05:00,5,Group_2,0.000000,0.000000,0.000000,0.000000,22.80760,NaN,NaN,...,NaN,-0.002021,-1.550219,4.119275,-0.469757,-1.668913,0.0,0.0,0.002021,0.0
46668,2021-09-09 12:05:00,6,Group_3,0.189700,0.000000,0.055600,0.000000,24.75276,NaN,NaN,...,NaN,0.306832,0.749880,48.191880,4.661130,64.331760,55.0,54.0,-0.306832,109.0
46669,2021-09-09 12:05:00,7,Control,0.257000,0.000000,0.063300,0.000000,29.16107,NaN,NaN,...,NaN,0.351710,0.799856,58.190076,4.501084,72.913500,18.0,57.0,-0.351710,75.0
46670,2021-09-09 12:05:00,8,Group_3,0.020000,0.000000,0.020000,0.000000,21.82529,NaN,NaN,...,NaN,0.276330,0.763900,44.092632,5.022097,57.742548,3.0,28.0,-0.276330,31.0


In [69]:
def incal_create_df_incal_format(df, dict_groups):
    categories_subjects = flat_list(list(dict_groups.values()))
    categories_groups = list(dict_groups.keys())

    date_time_level = pd.Series((pd.DatetimeIndex(df['Date_Time_1'])),
                                name='Date_Time_1')
    subjects_level = pd.Series(pd.Categorical(df['subjectID'],
                                              categories=categories_subjects,
                                              ordered=True),
                               name='subjectsID')
    group_level = pd.Series(pd.Categorical(df['Group'],
                                           categories=categories_groups,
                                           ordered=True),
                            name='Group')

    df = df.drop(columns=['Date_Time_1', 'subjectID', 'Group'])

    multi_index_dataframe = pd.concat(
        [date_time_level, subjects_level, group_level], axis=1)

    return pd.DataFrame(df.values,
                        index=pd.MultiIndex.from_frame(multi_index_dataframe),
                        columns=df.columns.values.tolist())

In [70]:
df = incal_create_df_incal_format(df, dict_groups)
df

actual_allmeters  actual_foodupa  \
Date_Time_1         subjectsID Group                                       
2021-07-28 16:45:00 1          Control          0.820163        0.000000   
                    10         Control          1.404414        0.313275   
                    11         Group_3          3.553978        0.006243   
                    12         Group_2         11.915580        0.204500   
                    13         Control          3.425906        0.116263   
...                                                  ...             ...   
2021-09-09 12:05:00 5          Group_2          0.000000        0.000000   
                    6          Group_3          0.189700        0.000000   
                    7          Control          0.257000        0.000000   
                    8          Group_3          0.020000        0.000000   
                    9          Group_2          0.043500        0.000000   

                                        actual_pedmeters  actual_waterupa  \
Date_Time_1         subjectsID Group                                        
2021-07-28 16:45:00 1          Control          0.259127         0.000000   
                    10         Control          1.140263         0.000000   
                    11         Group_3          3.171110         0.002538   
                    12         Group_2         11.116600         0.001420   
                    13         Control          2.752747         0.077571   
...                                                  ...              ...   
2021-09-09 12:05:00 5          Group_2          0.000000         0.000000   
                    6          Group_3          0.055600         0.000000   
                    7          Control          0.063300         0.000000   
                    8          Group_3          0.020000         0.000000   
                    9          Group_2          0.008400         0.000000   

                                        bodymass  envirolightlux  \
Date_Time_1         subjectsID Group                               
2021-07-28 16:45:00 1          Control  24.46564             NaN   
                    10         Control  22.61497             NaN   
                    11         Group_3  21.34998             NaN   
                    12         Group_2  24.69681             NaN   
                    13         Control  25.82226             NaN   
...                                          ...             ...   
2021-09-09 12:05:00 5          Group_2  22.80760             NaN   
                    6          Group_3  24.75276             NaN   
                    7          Control  29.16107             NaN   
                    8          Group_3  21.82529             NaN   
                    9          Group_2  27.65433             NaN   

                                        envirooccupancy  envirorh  \
Date_Time_1         subjectsID Group                                
2021-07-28 16:45:00 1          Control              NaN       NaN   
                    10         Control              NaN       NaN   
                    11         Group_3              NaN       NaN   
                    12         Group_2              NaN       NaN   
                    13         Control              NaN       NaN   
...                                                 ...       ...   
2021-09-09 12:05:00 5          Group_2              NaN       NaN   
                    6          Group_3              NaN       NaN   
                    7          Control              NaN       NaN   
                    8          Group_3              NaN       NaN   
                    9          Group_2              NaN       NaN   

                                        envirosound  envirotemp   kcal_hr  \
Date_Time_1         subjectsID Group                                        
2021-07-28 16:45:00 1          Control          NaN         NaN  0.407835   
                    10         Cont

# Shani exp: 
What is the effect of time restricted eating on the food intake, EE, EB 

Columns needed to analyze for this Q:
kcal_hr, actual_food_upa, bodymass

Calculeted parmeter EB:
kcal - food_intake = EB  

Intersting to see:
1. Cumuletive time series graph for each of our parms
2. Averages of start and end of bodymass in exp
3. Find outliers in kcal and food intake and get new averages - 2sd and boxplot
4. See reggrestion plot to visual the interaction or bodymass effect if any
4. Histograma to check normality 
5. Make a statical analysis: ancova or anova with interaction on kcal_hr and food_intake